In [1]:
from ppclass import pp
import apss_lib
import numpy as np
#%matplotlib inline

2020-10-02 17:24:14.530774


In [2]:
dadroplim = 0.2
dadroplim = 0.3
dadroplim = 0.35

In [3]:
case = "25M_LS300_W10"
case = "25M_LS300_W20"
case = "25M_LS120_W10"
case = "25M_LS120_W20"
case = "25M_LS30_W10"
case = "25M_LS30_W20"
case = "25M_LS0_W10"
case = "25M_LS0_W20"

In [4]:
!mkdir {case}
!mkdir {case}/pdf_sol_to_sol
!mkdir {case}/txt_sol_to_sol
!mkdir {case}/txt_per_sol
!mkdir {case}/pdf_stats

mkdir: impossible de créer le répertoire «25M_LS0_W20»: Le fichier existe
mkdir: impossible de créer le répertoire «25M_LS0_W20/pdf_sol_to_sol»: Le fichier existe
mkdir: impossible de créer le répertoire «25M_LS0_W20/txt_sol_to_sol»: Le fichier existe
mkdir: impossible de créer le répertoire «25M_LS0_W20/txt_per_sol»: Le fichier existe
mkdir: impossible de créer le répertoire «25M_LS0_W20/pdf_stats»: Le fichier existe


In [5]:
nn = 24 #8 #5
dim = 24 #48
#nn = 10
xxtab = np.random.rand(nn)*dim
yytab = np.random.rand(nn)*dim

xxtab = np.arange(24) ; yytab = np.arange(24)

print xxtab
print yytab

#dim=5 #20
#xxtab = np.arange(dim)
#yytab = np.arange(dim)



[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]
[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [6]:



sol = 9000
soltab = []



for xx in xxtab:
    xx = int(xx)
    print xx    
    for yy in yytab:
        yy = int(yy)

#hh = 144
##hh = 24*24 #576
#for hhh in range(hh):

        xx = int(24.*np.random.rand(1))
        yy = int(24.*np.random.rand(1))
        #print xx,yy

        zefile = "PSFC_LMD_LES_"+case
        ff = pp(file=zefile+"_every20.nc",var="PSFC",x=xx,y=yy,z=0).getf()

        ltstpp = 7. + np.arange(len(ff))/3700.
        utcpp = np.empty_like(ltstpp,dtype="str")
        for iii in range(len(utcpp)):
            utcpp[iii] = str(iii)
        
        #ltstpp = [str(ltst) for ltst in ltstpp]
        #utcpp = ltstpp
        
        
        dpp, spp = apss_lib.detrendsmooth(ff,1000)
        #dpp = ff - np.mean(ff) ; spp = np.mean(ff)
        
        indices, ddcount, drop, search = apss_lib.studypressure(sol,dpp,"",utcpp,ltstpp,\
                                                                droplim = -dadroplim,\
                                                                winsearch = 30,\
                                                                zefold=case,\
                                                                ltnum=True,isplot=False)
        
        #import ppplot ; pl = ppplot.plot1d() ; pl.f = dpp ; pl.x = ltstpp ; pl.fmt = '%.1f' ; pl.makeshow()
        soltab = np.append(soltab,sol)
        sol = sol+1        

        #print np.min(drop)

        
#import ppplot
#ppplot.plot1d(dpp)
#ppplot.show()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23


In [7]:
#nn = 25
#nn = 5
#nn = 10
#nn = 20
#nn = 48

In [8]:
#sol = 0
#
#
#
#for xx in range(nn):
#    print xx
#    for yy in range(nn):
#        #print xx,yy
#              
#        ff = pp(file="PSFC_LMD_LES25m_every10.nc",var="PSFC",x=xx,y=yy,z=0).getf()
#
#        ltstpp = 8. + np.arange(len(ff))/3700.
#        utcpp = np.empty_like(ltstpp,dtype="str")
#        for iii in range(len(utcpp)):
#            utcpp[iii] = str(iii)
#        dpp = ff - np.mean(ff)        
#
#        indices, ddcount, drop, search = apss_lib.studypressure(sol,dpp,"",utcpp,ltstpp,ltnum=True,isplot=False)
#        sol = sol+1        
#
#lastsol = sol
#        
##import ppplot
##ppplot.plot1d(dpp)
##ppplot.show()

In [9]:
### create event structured arrays
dtype = np.dtype([('drop',float), ('ltst',float), ('sol',int), ('utc','S23'), ('ratio',float)])
events = np.array([],dtype=dtype)

#np.arange(sol)

#print soltab

for iii in np.arange(len(soltab)):
    
    #print soltab[iii]
      
    #fifi, output = apss_lib.namefile(iii,"")
    fifi, output = apss_lib.namefile(soltab[iii],"")
    
    #print output
    
    ddfile = "./"+case+"/txt_per_sol/"+output+"DD_3.txt"
    
    ltst = np.loadtxt(ddfile,unpack=True,dtype=str,usecols=(2))   
    drop = np.loadtxt(ddfile,unpack=True,usecols=(1))
    utc = np.loadtxt(ddfile,unpack=True,usecols=(3),dtype='S23')
       
    nnn = drop.size

    if nnn > 1:
        for n in np.arange(nnn):
            element = np.array((drop[n],ltst[n],iii,utc[n],1),dtype=dtype)
            events = np.append(events,element)
    else:
        pass
        #print drop,ltst,soltab[iii],utc
        #events = np.append(events,np.array((drop,ltst,soltab[iii],utc,1),dtype=dtype))

print events.size




7614


In [10]:
### ordered file
oevents = np.sort(events, order='drop')
dafile = open("./"+case+"/alldrop_ordered.txt","w")
dafile.write('%6s ; %6s ; %3s ; %23s ; %5s\n' % ("_DROP_","_LTST_","SOL","YYYY-MM-DDTHH:MM:SS.sss","RATIO"))
for ooo in oevents:
    dafile.write('%6.3f ; %6.3f ; %03d ; %s ; %5.3f\n' % (ooo['drop'],ooo['ltst'],ooo['sol'],ooo['utc'],ooo['ratio']))
dafile.close()

In [11]:
print "MIN,MAX LTST",np.min(events['ltst']),np.max(events['ltst'])
print "MIN,MAX DROP",np.min(events['drop']),np.max(events['drop'])  

################# LT HISTOGRAM
limdrop = 0.5
limdrop = 0.3
limdrop = dadroplim
#limdrop = 2.0
nbins = 18
import matplotlib.pyplot as mpl
import ppplot
ppplot.changefont(18)
mpl.figure(figsize=(12,6))
##############################################
## drops stronger than limdrop
## + complete sols
idx = (events['drop'] < -limdrop)*(events['ratio'] > 0.99)
w = np.where(idx)
what_hist = events['ltst'][w]
##############################################
ppplot.figuref(x=16,y=6)
dabins = np.linspace(8,17,nbins+1)
#dabins = 'auto'
n, bins, patches = mpl.hist(what_hist,bins=dabins,density=True)
foo = mpl.ylabel("Probability density")
foo = mpl.xlabel("Mars LTST (hours)")
foo = mpl.xticks(bins)
foo = mpl.title("%i pressure drops stronger than %.2f Pa" % (what_hist.size, limdrop))
foo = mpl.xlim(xmin=8,xmax=17)

print "MEAN = ",np.mean(what_hist)
print "STD = ",np.std(what_hist)
#from scipy.stats import norm
#f_x = norm.pdf(bins, np.mean(what_hist), np.std(what_hist))
#x = bins
#mpl.plot(x,f_x)
####
##from scipy.stats import poisson
##X = np.arange(6,19)
##mpl.plot( X, poisson.pmf(X,np.mean(what_hist)), 'r-' )
####
ppplot.save(mode="pdf",filename="ltst_hist",folder='./'+case+'/pdf_stats/')

MIN,MAX LTST 8.991891891891893 16.546216216216216
MIN,MAX DROP -4.541 -0.345
MEAN =  12.80543383963674
STD =  1.3014538345516793


In [12]:
############################
limdrop = 0.5
limdrop = 1.0
#limdrop = 0.3
limdrop = dadroplim
#limdrop = 0.4
#limdrop = 1
############################

dafile = open("allDD_1.txt","w")
for drop in events['drop']:
  dafile.write('%5.3f ; %5.3f ; %5.3f ; %5.3f ; %5.3f ; %5.3f\n' % (10,10,-drop,10,10,10))
dafile.close()

bintab = [7,12,30,50]
bintab = [12,30,50]

from ddhist import histodd
for nb in bintab:
    histodd("allDD",drop=True,limrest=None,limtime=None,limwind=None,nbins=nb,limdrop=limdrop,\
            folder="./"+case+"/pdf_stats/")

### limrest: restrict limit (multiple of dx) -- greater equal
### limtime: use only data earlier than this local time 
### limdrop: use only data with deeper drop than value -- 0.5 to avoid false positives
### limwind: use only data with friction velocity larger than this value


/home/aspiga/soft/conda/miniconda2/envs/py2/lib/python2.7/site-packages/matplotlib/axes/_axes.py:6462: UserWarning: The 'normed' kwarg is deprecated, and has been replaced by the 'density' kwarg.
  warnings.warn("The 'normed' kwarg is deprecated, and has been "


In [13]:
def create_file_drop(soltab,file="test.txt",limdrop=0.0):
  dafile = open(file,"w")
  dafile.write('SOL RATIO N_D MAXdP\n')

  for dasol in soltab:
    fifi, output = apss_lib.namefile(dasol,"")
    ddfile = "./"+case+"/txt_per_sol/"+output+"DD_3.txt"
    drop = np.loadtxt(ddfile,unpack=True,usecols=(1))
    selectdrop = drop[drop <= -limdrop]
    ndrop = len(selectdrop)
    if ndrop > 0:
      extdrop = np.min(selectdrop)
    else:
      extdrop = 0.
    daind = np.where(sol == dasol)
    daratio = 1.
    
    try:
        dafile.write( '%03d %5.3f %03d %4.2f\n' % (dasol,daratio,ndrop,extdrop))
    except:
        print dasol
        print dasol,daratio,ndrop,extdrop
  dafile.close()
  return

#soltab = np.arange(lastsol)
create_file_drop(soltab,file="./"+case+"/txt_sol_to_sol/solDD_above_0p3.txt",limdrop=dadroplim)
create_file_drop(soltab,file="./"+case+"/txt_sol_to_sol/solDD_above_0p5.txt",limdrop=0.5)
create_file_drop(soltab,file="./"+case+"/txt_sol_to_sol/solDD_above_1p0.txt",limdrop=1.0)
create_file_drop(soltab,file="./"+case+"/txt_sol_to_sol/solDD_above_2p0.txt",limdrop=2.0)

In [14]:
## corrected variability
nx = 16 #32 #24 #16
ny = 4 #6
lim = 0.99 # to select regular (complete) sols

import numpy as np
import ppplot
ppplot.changefont(16)

stab = ["above_0p3","above_0p5","above_1p0","above_2p0"]
#stab = ["above_0p3"]

for sss in stab:

  sol, ratio, ndd, extdrop = np.loadtxt("./"+case+"/txt_sol_to_sol/solDD_"+sss+".txt",unpack=True,skiprows=1)
  fig = ppplot.figuref(x=nx,y=ny)
  pl = ppplot.plot1d(fig=fig)
  pl.xmin = sol[0]  - 1.0
  pl.xmax = sol[-1] + 1.0
  pl.nxticks = 10
  pl.ymin = 0
  pl.ymax = np.max(ndd/ratio)+1
  pl.nyticks = np.min([pl.ymax - pl.ymin + 1, 10])
  pl.fmt = '%.0f'
  pl.xlabel = "InSight sol"
  pl.ylabel = r'# pressure drops' #per hour'
  pl.linestyle = ""
  pl.marker = "s"
  pl.title = "pressure drops " + sss.replace("p",".").replace("_"," ") + " Pa"

  ##########################
  ## regular points first
  w = np.where(ratio >= lim)
  pl.f = ndd[w]/ratio[w]
  pl.x = sol[w]
  pl.color = "b"
  pl.make()

  ppplot.save(filename="drop_dtdvar_"+sss,mode="pdf",folder="./"+case+"/pdf_sol_to_sol/")

